In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import logging
import sys
import math
import json
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
warnings.filterwarnings('ignore')
from scipy.ndimage import uniform_filter1d
from etas import set_up_logger
from etas.inversion import round_half_up
from etas.simulation import generate_catalog
from filter_catalog_incompleteness import find_mainshock_times, apply_detection_threshold
sys.path.append('..')

from plot_utils_China import azimuthal_equidistant_projection, maxc,\
load_CN_borders,load_CN_faluts,plot_faluts,plot_borders


# Custom Colormap
colors = ["#5F0F40", "#C9DAEA", "#84A07C", "#E36414", "#39A9DB", "#0081A7", "#284B63", "#FFD449"]

In [7]:
polygon_coords=np.load('/home/tianweixi/EarthquakeNPP_CSEP_China/Datasets/ETAS/data/CSEP_China_Region.npy')
    # polygon_coords = polygon_coords[0:12962, [1, 0]] #点文件就是有问题，若是用全部的点文件会导致很久都跑不出来
polygon_coords = polygon_coords[:,[1,0]]
np.save('/home/tianweixi/EarthquakeNPP_CSEP_China/Datasets/ETAS/data/CSEP_China_Region_new.npy',polygon_coords)


# Run etas-simulation 

In [ ]:
## The time for CSEP_China region is about 30 seconds
# We will get the simulation catalog named ETAS_China_catalog here!
from simulate_catalog import run_simulation
run_simulation()

### Lets read the synthetic catalog and visualise it

In [ ]:
with open("/home/tianweixi/EarthquakeNPP_CSEP_China/Datasets/ETAS/simulate_ETAS_California_catalog_config.json", 'r') as f:
        config = json.load(f)

catalog = pd.read_csv(
                config["fn_store"],
                index_col=0, #表明将文件的第0列当作索引而不是普通列包含再数据中，感觉不是很有必要
                parse_dates=["time"], #将指定的列自动转换为datetime格式，便于后续处理
                dtype={"url": str, "alert": str},#将指定的列转换为字符串，但是这个目录文件里并没有，所以也是多余的
            )

catalog = catalog.sort_values(by='time')
catalog.head()

In [ ]:
%matplotlib inline
plt.figure(figsize=(15, 7))
plt.gca().set_facecolor((0.95, 0.95, 0.95))

z = (9.5**catalog['magnitude'])*0.0005
plt.scatter(catalog['time'],catalog['magnitude'],s=z,color =colors[3])
plt.xlabel('Time',fontsize=12)
plt.ylabel('Magnitude',fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylim([2.2,7.5])
plt.show()

Since ETAS uses the great-circle distance between two points on a sphere (km), for the NPPs we need to project the coordinates of the events into a space such that inter-event distances are in km. To do this, we can use the Azimuthal equidistant projection [2], which preserves distances from all points to a center point.

We pass the mean coordinates of the catalog as the center coordinates of the projection.

In [6]:
center_latitude = catalog['latitude'].mean()  # Latitude of the center point
center_longitude = catalog['longitude'].mean()  # Longitude of the center point

# Convert latitude and longitude to Cartesian coordinates using Plate Carrée projection
catalog['x'], catalog['y'] = azimuthal_equidistant_projection(catalog['latitude'], catalog['longitude'], center_latitude, center_longitude)

In [ ]:
catalog.head()
catalog.to_csv("/home/tianweixi/EarthquakeNPP_CSEP_China/Datasets/ETAS/data/ETAS_simulation_catalog/ETAS_China_catalog_Azimuthal_euqidisyant_projection.csv")

# Visualise the catalog

In [ ]:
import os
catalog = pd.read_csv('/home/tianweixi/EarthquakeNPP_CSEP_China/Datasets/ETAS/data/ETAS_simulation_catalog/ETAS_China_catalog_Azimuthal_euqidisyant_projection.csv')
catalog['time'] = pd.to_datetime(catalog['time'])


%matplotlib inline
plt.figure(figsize=(15, 7)) # the size of figure
plt.gca().set_facecolor((0.95, 0.95, 0.95)) # get current axis(gca), set the face color

# Compute M_c(t) across the filetred catalog
window_size=300
nwindows = math.floor(len(catalog['magnitude'])/window_size) #calculate the number of windows


Mc_t = [0]*nwindows
mid_time = [0]*nwindows

comp_T = catalog['time']
comp_M = catalog['magnitude']

for i in range(nwindows): #依次获取每个window包含的地震震级信息，并输入（震级，震级间隔）到maxc进行计算
    
    mid_time[i] =  pd.Timestamp(pd.Series(comp_T[i*window_size:(i+1)*window_size]).mean())

    window = comp_M[i*window_size:(i+1)*window_size] 
    Mc_t[i] = maxc(window,0.05)

# Smooth M_c(t) for plotting
Mc_t = uniform_filter1d(Mc_t, size=40)

# Plotting
plt.step(mid_time,Mc_t,colors[0],label=r'$M_c(t)$',lw=3)
z = (9.5**catalog['magnitude'])*0.0001
plt.scatter(catalog['time'],catalog['magnitude'],s=z,color =colors[3])
plt.xlabel('Time',fontsize=12)
plt.ylabel('Magnitude',fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.ylim(0,9.0)
# save the figure
os.makedirs("data/Figure", exist_ok=True)
plt.savefig('data/Figure/1.MC.png', dpi=300, format='png', bbox_inches='tight')

plt.show()

let's now visualise the coordinates of events in the catalog

In [ ]:
# Create a figure with Cartopy
fig = plt.figure(figsize=(10.5, 8.1))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator()) #添加底图并设置投影方式为mercator投影
max_lat_plot, min_lat_plot, max_lon_plot, min_lon_plot = 35, 20, 110, 95# 经纬度范围,这是绘图的范围,不太一样
ax.set_extent([min_lon_plot, max_lon_plot, min_lat_plot, max_lat_plot], crs=ccrs.PlateCarree()) #设定范围，并设定为平面等距投影，也就是平常所用的经纬度书韩剧就可以


#Read borders and faluts file
# 读取border数据
file_path = 'data/CN-border-La.gmt'
borders = load_CN_borders(file_path)
# 读取faluts数据
file_path = 'data/CN-faults.gmt'
faluts = load_CN_faluts(file_path)

# Add gridlines with customization
gl = ax.gridlines(draw_labels=True, color='gray', linestyle='--', alpha=0.4)
gl.right_labels = False
gl.top_labels = False
ax.tick_params(axis='x', labelsize=16) #设置刻度参数
ax.tick_params(axis='y', labelsize=16)

# Plot polygon
polygon_coords = np.load('data/CSEP_China_Region.npy')
polygon_coords = polygon_coords[0:12962, [1, 0]] #交换一下经纬度,同时这里只能使用直到12962的数据点.因为原始文件错误,导致后面进行地震目录过滤时报错交叉,通过121962则可以避免.
plot_polygon_coords = np.append(polygon_coords, [polygon_coords[0]], axis=0)
ax.plot(plot_polygon_coords[:, 1], plot_polygon_coords[:, 0], transform=ccrs.PlateCarree(), color='red', lw=2.0) #这里的plot()中第一个参数是经度，第二个是纬度

# Plot events
ax.scatter(catalog['longitude'], catalog['latitude'], transform=ccrs.PlateCarree(), s=0.4, color=colors[3], alpha=0.8)

# Add colored land and ocean, we need to plot them after plotting the event. Otherwise, it will be covered
ax.add_feature(cfeature.OCEAN.with_scale('50m'),facecolor="#0081A7")
ax.add_feature(cfeature.LAND.with_scale('50m'),facecolor="#5F0F40")

#绘制faluts和borders数据

plot_faluts(faluts,ax,min_lon_plot, max_lon_plot, min_lat_plot, max_lat_plot)
plot_borders(borders,ax)

#保存图片
plt.savefig('data/Figure/2.Event_and_Region.png', dpi=300, format='png', bbox_inches='tight')

plt.show()


## Generating the incomplete catalog

$$M_c(M,t) = M/2 - 0.25 - \log_{10}(t),$$
where $M$ is the mainshock magnitude. Events below this threshold are removed using mainshocks of Mw 5.2 and above.

In [ ]:
# Set the magnitude threshold for mainshock selection
magnitude_threshold = 5.2 #为什么

# Find the mainshock times
mainshock_times = find_mainshock_times(catalog, magnitude_threshold)

print('Number of Mainshocks:', len(mainshock_times))

# Apply detection threshold to aftershocks of each mainshock
filtered_catalog = apply_detection_threshold(catalog, mainshock_times)

print("Original number of events:", len(catalog))
print("Number of events after applying time and magnitude dependent detection threshold:", len(filtered_catalog))


We apply the Azimuthal equidistant projection as before and write to a file.

In [18]:
filtered_catalog.to_csv("/home/tianweixi/EarthquakeNPP_CSEP_China/Datasets/ETAS/data/ETAS_simulation_catalog/ETAS_China_incomplete_catalog.csv")

## This is the additional option and exercises for you to understand how to removed earthquakes

We can see how events have been removed under the line $M_c(M,t)$.
只是以其中一个地震为例子

In [ ]:
%matplotlib inline
plt.figure(figsize=(15, 7))
plt.gca().set_facecolor((0.95, 0.95, 0.95))


# Find the index of the largest earthquake by magnitude 
largest_earthquake_index = np.where(filtered_catalog['magnitude'].to_numpy()==filtered_catalog['magnitude'].max())[0][0]-6 #但是不懂这个-6是什么意思，前面的已经找到了最大的地震的index了(反正计算的时候是所有时间减去第6个值的索引)

# Get the 150 earthquakes preceding the largest one
#但是这里明明是计算的大地震后的150个的地震的索引
start_index = largest_earthquake_index
df_preceding = filtered_catalog.iloc[largest_earthquake_index:largest_earthquake_index+150]

# Compute time_dependent line
M = filtered_catalog['magnitude'].max()
time_days = (df_preceding['time'] - df_preceding['time'].iloc[6]).dt.total_seconds() / (24 * 3600) #计算各个地震和主震的时间差（转化成秒）
time_dependent = np.maximum(M / 2 - 0.25 - np.log10(time_days),2.5) #这里和最低下限进行了一个筛选，因为根据上面的公式计算，

# Plotting
z = (3.5**df_preceding['magnitude'])*0.4
plt.scatter(df_preceding['time'], df_preceding['magnitude'], color=colors[3], s=z)
plt.plot(df_preceding['time'], time_dependent, color=colors[0], linewidth=3,label=r'$M_c(M,t)$')
plt.xlabel('Time',fontsize=12)
plt.ylabel('Magnitude',fontsize=12)
plt.ylim([2.2,7.5])
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12)
plt.show()